### Word to Vector

Word2vec (word embedding)词用向量表示。分skip-gram 和CBOW 两种训练方法。数据量大时，skip-gram效果好。
Skip-gram 基于中心词生成背景词，CBOW 基于背景词生成中心词。

#### Skip-gram

Skip-gram 思路是相邻越近的两个词的相似度越高。越远的相似度越低。

- 近义词：两个词是近义词，则这两个词分别邻近的词会很相似，所以近义词的词向量会很相似。

- 二次采样：对于出现频率高的词，比如介词the，所有的名词都可能跟the 邻近，但这些名词词向量肯定大同小异，因此去掉高频词，训练出的词向量效果更好。

- 负采样：求解中心词出现的条件下背景词出现的概率，需要两个词向量的内积，除以中心词和其他所有词的内积之和，并做Softmax激活。这样feed propagation，back propagation都要计算 $O$(词典大小) 时间复杂度。引入负样本的思想，以窗口为界，中心词和窗口内词为正样本，中心词和窗口外词为负样本，只要计算和求导只要更新少量词的向量即可。

  负采样改变了目标函数，正、负样本分别做词向量内积，用Sigmoid激活之后求积。有点类似Logistic Regression里面交叉熵损失函数，去掉指数后的乘积。

    1. trick：按窗口大小随机取s = [1, window_size]，正样本一次取s个，让模型更关注距离中心词更近的词。
    2. 负样本选取：词出现频数的0.75 次幂作为权重，一个正样本，对词典随机选取K个负样本。减弱频次差异的影响，让小概率词被采样概率变大。